In [44]:
#Pré-requis
#pip install altair vega_datasets panel altair_saver bokeh jupyter_bokeh

# D'abord vers la jointure avec les données géographiques

# Aller chercher les données 
import pandas as pd
data = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vSRy3fIKwWXSRg4QI_A9sQRNmWKnsNSA-TpgMenVz0twxfRmJGiFKN8Q8VSpBwxNw/pub?gid=1499792138&single=true&output=csv", encoding='utf8')


data.set_index('Année',inplace = True)

zones_peche = data.T
zones_peche['maximum'] = zones_peche.max(axis=1)


zones_peche.reset_index(inplace=True, drop=False)
#print(zones_peche.columns)
zones_peche = zones_peche.rename(columns={'index': 'zone'})

#print(zones_peche.index)
#zones_peche.reset_index(drop=True, inplace=True)
#print(zones_peche)
print(zones_peche.columns)


## Faire la jointure avec les zones

Index([   'zone',      1763,      1764,      1765,      1766,      1767,
            1768,      1769,      1770,      1771,      1772,      1773,
            1774,      1775,      1776,      1777,      1778,      1779,
            1780,      1781,      1782,      1783,      1784,      1785,
            1786,      1787,      1788,      1789,      1790,      1791,
            1792, 'maximum'],
      dtype='object', name='Année')


Année,zone,1763,1764,1765,1766,1767,1768,1769,1770,1771,...,1784,1785,1786,1787,1788,1789,1790,1791,1792,maximum
0,Islande,81891,214218,411990,515301,470314,496928,547641,589933,541443,...,972825,1022839,823937,760300,842976,791887,628498,494656,730078,1999689
1,Shetlands,64269,81549,91178,49516,67666,71727,96073,68194,60004,...,73642,72902,67316,117006,78326,60367,61502,92339,230417,236127
2,Terre-Neuve,0,0,0,0,0,0,0,72563,106543,...,31680,143276,346964,384473,232956,211931,114353,124015,104362,384473
3,Dunkerque_mer_du_nor,355800,480560,361658,205002,373652,201899,179246,171089,167693,...,288467,289887,200955,103818,106486,115125,144756,114828,198860,480560


## Code pour faire la carte des zones de pêche : evolution du rapport année/max par zone, donne une intensite de pêche dans une carte chroroplèthe

### des petits bouts de code d'Altair

alt.Chart(exports_tomap).mark_circle().encode(
        longitude='x:Q',
        latitude='y:Q',
        size=alt.Size('{}:Q'.format(variable), title='Analyse de la variable :  {}'.format(variable), scale=alt.Scale(domain=[-1, 7000], range=[10, 400], clamp=True, zero= False)), 
        color=alt.value('orange'),
        tooltip=['toponyme_standard_fr:N','{}:Q'.format(variable)]
    )

    import altair as alt
from vega_datasets import data

counties = alt.topo_feature(data.us_10m.url, 'counties')
source = data.unemployment.url

alt.Chart(counties).mark_geoshape().encode(
    color='rate:Q'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(source, 'id', ['rate'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

In [1]:
import altair as alt
import panel as pn
pn.extension('vega')

#pays_url = './word1789_topojson.json'
pays_url = 'https://raw.githubusercontent.com/cplumejeaud/intoZcarto/main/python/word1789_topojson.json'
pays = alt.topo_feature(pays_url, 'word1789_geojson')

#print(zones_peche.columns)
# remote geojson data object
url_geojson = 'C:\Travail\ULR_owncloud\ANR_PORTIC\datasprint2_Dunkerque\Nantes\zones_peche_4326.json'
data_geojson_remote = alt.Data(url=url_geojson, format=alt.DataFormat(property='features',type='json'))

# chart object
alt.Chart(data_geojson_remote).mark_geoshape(
).encode(
    color="properties.name:N"
).project(
    type='identity', reflectY=True
)

variable = 'maximum' #'nb_ships_total', 'sum_tonnage_total', 'nb_ships2', 'sum_tonnage_per_product_total'
# World 1789, center on England
map = alt.layer(
    # use the sphere of the Earth as the base layer
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#e6f3ff'
    ),
    # add a graticule for geographic reference lines
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#ffffff', strokeWidth=1
    ),
    # and then the countries of the world
    alt.Chart(pays).mark_geoshape(
        fill='lightgray', stroke='white', strokeWidth=0.5
    ),
    #the zones de pêche : coloriser en fonction de la variable maximum
    alt.Chart(data_geojson_remote).mark_geoshape(
        fill='lightgray',
        stroke='white'
    )
    
).properties(
    width=500,
    height=600,
    title='Evolution de l''intensité de la pêche dans les principales zones de pêche de Dunkerque'
).project('conicEquidistant', scale=1000).configure_projection(center = [-5, 47]).configure_view(stroke=None)

map.save('map.json')
map.save('map_export_Dunkerque.html')

map

NameError: name 'exports_tomap' is not defined